In [1]:
import pandas as pd
import matplotlib.pyplot as plt

original_apple_data = pd.read_csv('AppleStore.csv')
original_google_data = pd.read_csv('googleplaystore.csv')
print('the number of original Apple store dataset:', len(original_apple_data))
print('the number of original google play store dataset:', len(original_google_data))

the number of original Apple store dataset: 7197
the number of original google play store dataset: 10841


In [ ]:
# print(original_apple_data.dtypes)
# print(original_google_data.dtypes)
print('Missing values in Google play store\n{}'.format(original_google_data.isnull().sum()))
print('Missing values in Apple app store\n{}'.format(original_apple_data.isnull().sum()))

In [ ]:
original_apple_data.price.value_counts()
original_google_data.Rating.value_counts()

In [ ]:
google_data = google_data[google_data.Rating < 5]
google_data.Rating.value_counts()
apple_data[apple_data.user_rating == 0]

In [ ]:
plt.figure();
google_data.Rating.plot.density()
plt.show()

In [ ]:
plt.figure();
original_apple_data.user_rating.plot.hist()
plt.show()

In [2]:
google_useful_attr = ['App', 'Rating', 'Reviews']
# , 'Size', 'Price', 'Genres', 'Current Ver']
print('Preprocessing google play store dataset...')
print('select column:', ', '.join(google_useful_attr), '\n')
google_data = original_google_data[google_useful_attr]
google_data = google_data.drop_duplicates(subset=['App'], keep='last')
print('After drop duplicates:', len(google_data))
google_data = google_data[google_data.Rating <= 5]
google_data = google_data[google_data.Rating >= 0]
google_data = google_data.replace({'Reviews': '3.0M'}, {'Reviews': '3000000'})
google_data.Reviews = google_data.Reviews.astype(int)
google_data = google_data[google_data.Reviews > 1000]
print('select review > 1000:', len(google_data))

Preprocessing google play store dataset...
select column: App, Rating, Reviews 

After drop duplicates: 9660
select review > 1000: 4802


In [3]:
apple_useful_attr = ['track_name', 'rating_count_tot', 'user_rating']
print('Preprocessing apple store dataset...')
print('select column:', ', '.join(apple_useful_attr), '\n')
apple_data = original_apple_data[apple_useful_attr]
apple_data = apple_data[apple_data.user_rating >= 0]
apple_data = apple_data[apple_data.user_rating <= 5]
apple_data = apple_data[apple_data.rating_count_tot > 100]
print('select review > 1000:', len(apple_data))

Preprocessing apple store dataset...
select column: track_name, rating_count_tot, user_rating 

select review > 1000: 4489


In [ ]:
apple_name = set(apple_data.track_name)
google_name = set(google_data.App)
apple_name_count = {}
for name in apple_name:
    cur_name = name
    if ':' in cur_name:
        cur_name = cur_name.split(':')[0]
    if ' – ' in cur_name:
        cur_name = cur_name.split(' – ')[0]
    if ' - ' in cur_name:
        cur_name = cur_name.split(' - ')[0]
    if cur_name not in apple_name_count:
        apple_name_count[cur_name] = 0
    apple_name_count[cur_name] += 1

google_name_count = {}
for name in google_name:
    cur_name = name
    if ':' in cur_name:
        cur_name = cur_name.split(':')[0]
    if ' – ' in cur_name:
        cur_name = cur_name.split(' – ')[0]
    if ' - ' in cur_name:
        cur_name = cur_name.split(' - ')[0]
    if cur_name not in google_name_count:
        google_name_count[cur_name] = 0
    google_name_count[cur_name] += 1

In [ ]:
same_name = set(apple_name_count.keys()).intersection(set(google_name_count.keys()))
len(same_name)

In [4]:
google_data = google_data.rename(index=str, columns={'App': 'google_name'})
apple_data = apple_data.rename(index=str, columns={'track_name': 'apple_name'})
google_data.insert(loc=0, column='name', value=google_data.google_name)
apple_data.insert(loc=0, column='name', value=apple_data.apple_name)

In [5]:
def x(name):
    cur_name = name
    if ':' in cur_name:
        cur_name = cur_name.split(':')[0]
    if ' – ' in cur_name:
        cur_name = cur_name.split(' – ')[0]
    if ' - ' in cur_name:
        cur_name = cur_name.split(' - ')[0]
    return cur_name
apple_data.name = apple_data.name.apply(x)
google_data.name = google_data.name.apply(x)

In [11]:
google_data = google_data.rename(index=str, columns={'Rating': 'google_rating'})
apple_data = apple_data.rename(index=str, columns={'user_rating': 'apple_rating'})
google_data = google_data.rename(index=str, columns={'Reviews': 'google_rating_count'})
apple_data = apple_data.rename(index=str, columns={'rating_count_tot': 'apple_rating_count'})

apple_data.drop_duplicates(subset=['name'], keep='last')
google_data.drop_duplicates(subset=['name'], keep='last')
res = pd.merge(apple_data, google_data, on='name')
res

In [17]:
len(res)

579

In [14]:
res.head()
res.apple_rating.corr(res.google_rating)
res.to_csv('result.csv')

In [ ]:
count = 0
count2 = 0
for name in apple_name:
    cur_name = name
    if ':' in cur_name:
        cur_name = cur_name.split(':')[0]
    if ' – ' in cur_name:
        cur_name = cur_name.split(' – ')[0]
    if ' - ' in cur_name:
        cur_name = cur_name.split(' - ')[0]
    if cur_name not in same_name:
        print(cur_name)
        count += 1
        for gname in google_name:
            if cur_name in gname:
                print(gname, '\n')
                count2 += 1
                break
print(count, count2)

In [ ]:
'–' == '-'
'-' == '-'
' ' == ' '
original_google_data[original_google_data.App.str.contains('Gmail')]

In [ ]:
'NBC – Watch Now and Stream Full TV Episodes'.split(' – ')[0]

In [ ]:
count3 = 0
count4 = 0
for name in google_name:
    cur_name = name
    if ':' in cur_name:
        cur_name = cur_name.split(':')[0]
    if ' – ' in cur_name:
        cur_name = cur_name.split(' - ')[0]
    if cur_name not in same_name:
#         print(cur_name)
        count3 += 1
        for aname in apple_name:
            if cur_name in aname:
#                 print(gname, '\n')
                count4 += 1
                break
print(count3, count4)

In [ ]:
for k,v in apple_name_count.items():
    if v > 1:
        pass
#         print(k, v)
for name in apple_name:
    if 'Cut the Rope' in name:
        print(name)
for name in google_name:
    if 'Cut the Rope' in name:
        print(name)

In [ ]:
apple_name_count = {}
for name in apple_name:
    cur_name = name
    if ':' in cur_name:
        cur_name = cur_name.split(':')[0]
    if ' - ' in cur_name:
        cur_name = cur_name.split(' - ')[0]
        if cur_name not in apple_name_count:
            apple_name_count[cur_name] = 0
        apple_name_count[cur_name] += 1

In [ ]:
for name in apple_name:
    cur_name = name
    if ':' in cur_name:
        cur_name = cur_name.split(':')[0]
    if ' - ' in cur_name:
        cur_name = cur_name.split(' - ')[0]
        if apple_name_count[cur_name] == 1:
            name = cur_name

In [ ]:
google_name_count = {}
for name in google_name:
    cur_name = name
    if ':' in cur_name:
        cur_name = cur_name.split(':')[0]
    if ' - ' in cur_name:
        cur_name = cur_name.split(' - ')[0]
        if cur_name not in google_name_count:
            google_name_count[cur_name] = 0
        google_name_count[cur_name] += 1
        
for k,v in google_name_count.items():
    if v > 1:
        print(k)

In [ ]:
google_name_count = {}
for i in google_name:
    if ':' in i:
        new_name = i.split(':')[0]
        if new_name not in google_name_count:
            google_name_count[new_name] = 0
        google_name_count[new_name] += 1
for k,v in google_name_count.items():
    if v>1:
        print(k)

In [ ]:
apple_name_count = {}
for i in apple_name:
    if ' - ' in i:
        new_name = i.split(' - ')[0]
        if new_name not in apple_name_count:
            apple_name_count[new_name] = 0
        apple_name_count[new_name] += 1

In [ ]:
apple_name_count = {}
count = 0
for i in google_name:
    if ' - ' in i and ':' in i:
        count+=1
count

In [ ]:
len(apple_name.intersection(google_name))

In [ ]:
len(google_name)

In [ ]:
len(apple_data[apple_data.rating_count_tot > 10000])

In [ ]:
len(google_data[google_data.Reviews > 500000])

In [ ]:
apple_data = apple_data[apple_data.rating_count_tot > 10000]
google_data = google_data[google_data.Reviews > 500000]

In [ ]:
apple_name = set(apple_data.track_name)
google_name = set(google_data.App)
apple_name.intersection(google_name)

In [ ]:
apple_name

In [ ]:
google_name.difference(apple_name)

In [ ]:
temp = {}
for i in apple_name:
    if ':' in i:
        new_name = i.split(':')[0]
        if new_name in temp:
            temp[new_name] += 1
        else:
            temp[new_name] = 1
temp